In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from nessai.flowmodel import FlowModel
from scipy.stats import norm
import torch

from thesis_utils.colours import pillarbox
from thesis_utils.plotting import pp_plot, set_plotting, save_figure
from thesis_utils.random import seed_everything

seed_everything()
set_plotting()

# Training data

In [ ]:
data = 5 * np.random.randn(10_000, 2)

In [ ]:
target_dist = norm(loc=2, scale=2)

In [ ]:
weights = target_dist.pdf(data).prod(axis=1) / norm(scale=5).pdf(data).prod(
    axis=1
)

In [ ]:
true = target_dist.rvs(size=(10_000, 2))

## Flow

In [ ]:
config = dict(
    annealing=False,
    patience=50,
    lr=0.001,
    batch_size=1000,
    model_config=dict(
        n_inputs=2,
        n_neurons=16,
        n_blocks=2,
        kwargs=dict(
            batch_norm_between_layers=False,
            batch_norm_within_layers=False,
            # linear_transform="lu",
        ),
    ),
)

In [ ]:
fm = FlowModel(config=config, output="outdir/training_w_weights/")

In [ ]:
history = fm.train(data, weights=weights)

In [ ]:
plt.plot(history["loss"])
plt.plot(history["val_loss"])
plt.show()

In [ ]:
with torch.inference_mode():
    samples = fm.sample(5_000)

In [ ]:
fig = plt.figure()
plt.scatter(data[:, 0], data[:, 1])
plt.scatter(true[:, 0], true[:, 1])
plt.scatter(samples[:, 0], samples[:, 1])
plt.show()

In [ ]:
x = np.linspace(-20, 20, 1000, endpoint=True)
target_pdf = target_dist.pdf(x)
data_pdf = norm(loc=0, scale=5).pdf(x)

In [ ]:
labels = [r"$x_0$", r"$x_1$"]
colours = ["C0", "C1"]

fig, axs = plt.subplots(2, 1)
axs[0].plot(x, target_pdf, label="Target", c="k")
axs[0].plot(x, data_pdf, label="Training data", ls="--", c=pillarbox)
axs[0].hist(
    samples[:, 0],
    32,
    density=True,
    histtype="step",
    ls="-.",
    label=r"$x_0$",
    color=colours[0],
)
axs[0].hist(
    samples[:, 1],
    32,
    density=True,
    histtype="step",
    ls="-.",
    label=r"$x_1$",
    color=colours[1],
)
axs[0].set_xlabel(r"$x_i$")
axs[0].set_ylabel(r"$p(x_i)$")

axs[0].legend()

n_steps = 1000
for d, label, colour in zip(samples.T, labels, colours):
    sorted_data = np.sort(d)
    pp_data = target_dist.cdf(sorted_data)
    fig = pp_plot(
        pp_data,
        labels=label,
        ax=axs[1],
        n_steps=n_steps,
        confidence_intervals=False,
        colours=colour,
    )

axs[1].legend()
axs[1].set_xlabel("Theoretical percentiles")
axs[1].set_ylabel("Sample percentiles")

plt.show()
save_figure(fig, "flow_weights_plot.pdf", "figures")
plt.show()